In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 문서 가져오기
pdf_path = "../data/Sustainability_report_2024_kr.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()
len(docs)

83

In [ ]:
# 2. 청크 나누기
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)
chunks = splitter.split_documents(docs[:20])
print(len(chunks))

48


In [ ]:
# 3. 벡터 스토어
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate

embeddings = OpenAIEmbeddings()
db_path = "../vectorStore/rag_eval_20"
vectorStore = Chroma.from_documents(
    documents = chunks,
    embedding = embeddings,
    persist_directory = db_path,
    collection_name = "samsung2024_eval"
)

In [9]:
# 4. retriever 구성하기
retriever = vectorStore.as_retriever(
    search_kwargs = {"k" : 5}
)

In [5]:
# 5. prompt 구성하기
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are a helpful assistant. Answer strictly based on the provided context."
    "If the answer is not in the context, say you don't know."
    "context : {context}"
)
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])

In [6]:
# 6. 모델 구성하기
model = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0
)

In [7]:
# 7. outputParser
from langchain_core.output_parsers import StrOutputParser
outputparser = StrOutputParser()

In [ ]:
# 8. chain 설정
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# 문서 합치는 함수 설정
def format_docs(docs):
    return "\n\n---\n\n".join(d.page_content for d in docs)

# chain 만들기
rag_chain = (
    {"context" : RunnableLambda(lambda x : x["question"]) | retriever | format_docs,
    # {"context" : retriever | format_docs, -> 이렇게 하면 invoke 할때 (key-value가 아닌 value값만 넣으면 됨 -> rag_chain.invoke("삼성전자의 전망은?"))
     "question" : RunnablePassthrough()
    }
    | rag_prompt
    | model
    | outputparser
)

In [12]:
rag_chain.invoke({"question" : "삼성전자의 전망은?"})

'삼성전자는 어려운 대내외 환경 속에서도 지속 성장의 기반 마련을 위해 역대 최고 수준의 연구개발 투자(28.3조원)와 전략적 시설투자(53.1조원)를 통해 기술 리더십을 강화하며 중장기 수요에 미리 대응하고 있습니다. 또한, 지속가능경영을 기업의 방향성과 사업경쟁력, 기술혁신의 원동력으로 삼아 새로운 도약을 모색하고 있으며, 환경, 사회, 경제적 리스크와 지정학적 불확실성 속에서도 지속가능성을 기반으로 더욱 발전하는 모습을 통해 이해관계자들의 기대에 부응하기 위해 최선을 다할 계획입니다.'

In [ ]:
# 9. 평가용 데이터 불러오기
import pandas as pd
excel_path = "report_2024_test.xlsx"
df = pd.read_excel(excel_path)
df.head()

,user_input,reference_contexts,reference,synthesizer_name
0,Why 독일 supply chain due diligence law importan...,"['CEO 메시지\nMessage from \nOur CEO\n주주, 고객, 협력회...",독일에서는 공급망의 인권과 근로환경 관리를 의무화하는 공급망실사법이 2023년 발효...,single_hop_specific_query_synthesizer
1,삼성전자의 지속가능경영 전략과 주요 이슈 관리는 어떻게 이루어지고 있나요?,"[""접수된 고충의 처리 원칙에 대한 기준을 수립하였고, 공급망 관리에 \n있어서는 ...",삼성전자는 지속가능경영보고서를 통해 글로벌 공시규제 프레임워크에 맞춰 주요 지속가능...,single_hop_specific_query_synthesizer
2,What Samsung Global Code of Conduct mean for c...,['삼성전자 지속가능경영보고서 2024\n05\nOur Company Appendi...,Samsung Electronics established five core valu...,single_hop_specific_query_synthesizer
3,What was the global operational footprint of S...,['Our Company AppendixMateriality Assessment F...,"By the end of 2023년, Samsung Electronics had a...",single_hop_specific_query_synthesizer
4,What are the 2023 financial results for the ma...,['주요 사업부\n Device eXperienceDX\nDS Device Solu...,The sales and operating profit figures for the...,single_hop_specific_query_synthesizer


- user_input : 질문
- retrieved_contexts : 예상되는 답변을 만들기위해 참고한 contexts
- reference : 예상되는 답변
***
- user_input : 질문
- retrieved_contexts : 검색한 자료 -> 리스트형태
- response : 실제 답변

In [ ]:
# for문으로 구현하기
contexts = []
for question in df["user_input"]:
    docs = retriever.invoke(question)
    context_list = []
    for d in docs:
        context_list.append(d.page_content)
    contexts.append(context_list)

In [ ]:
contexts

In [ ]:
answer = []
for question in df["user_input"]:
    an = rag_chain.invoke({"question" : question})
    answer.append(an)

In [17]:
answer[:5]

["The provided context does not contain specific information about the 독일 (German) supply chain due diligence law or its importance for sustainability management in a company. Therefore, I don't know the answer based on the given context.",
 "삼성전자의 지속가능경영 전략과 주요 이슈 관리는 다음과 같이 이루어지고 있습니다.\n\n1. 최고 의사결정기구인 이사회와 지속가능경영위원회가 DX부문의 기후변화, 자원순환 등 환경 분야를 경영의 핵심 영역으로 인식하고 이를 관장합니다. 이사회는 환경경영전략과 목표를 승인하고 주요 활동을 감독합니다.\n\n2. 2022년에는 중장기 기후변화 대응 및 자원순환 목표를 포함하는 삼성전자 新환경경영전략이 결의되었으며, 2023년에는 '新환경경영전략 1주년 주요성과'가 보고되었습니다.\n\n3. DX부문 CEO가 환경경영전략 수립, 이행 과제 발굴, 투자 실시 등 주요 사안에 대한 책임과 권한을 가지고 있으며, 각 사업부장 및 기능 부서장들과 함께 전사 지속가능경영협의회를 운영합니다.\n\n4. 환경 분야 임원들로 구성된 환경경영TF에서는 환경경영 계획을 수립하고 이행 성과를 점검합니다.\n\n5. 수립된 환경경영 계획은 지속가능경영추진 센터, Global EHS 실, 지역별 환경 전담조직, 사업부 지속가능경영사무국 등이 담당하여 시행합니다.\n\n6. 2021년부터 온실가스 감축 실적을 조직 평가에 반영하기 시작했으며, 2023년부터는 재생에너지 전환, 고효율제품 개발, 폐기물 재활용 실적 등을 조직/임원 성과 평가에 추가했습니다. 2024년에는 수자원 관리 항목도 추가할 예정입니다.\n\n7. 이사회 산하 지속가능경영위원회는 사외이사들로 구성되어 분기별 1회 이상 환경경영을 포함한 지속가능경영 방향성과 이행 성과를 감독합니다.\n\n8. 

In [37]:
df["response"] = answer
df["retrieved_contexts"] = contexts
df.head()

,user_input,reference_contexts,reference,synthesizer_name,response,retrieved_contexts
0,Why 독일 supply chain due diligence law importan...,"[CEO 메시지\nMessage from \nOur CEO\n주주, 고객, 협력회사...",독일에서는 공급망의 인권과 근로환경 관리를 의무화하는 공급망실사법이 2023년 발효...,single_hop_specific_query_synthesizer,The provided context does not contain specific...,[삼성전자 지속가능경영보고서 2024\n18\nOur Company Appendix...
1,삼성전자의 지속가능경영 전략과 주요 이슈 관리는 어떻게 이루어지고 있나요?,"[접수된 고충의 처리 원칙에 대한 기준을 수립하였고, 공급망 관리에 \n있어서는 비...",삼성전자는 지속가능경영보고서를 통해 글로벌 공시규제 프레임워크에 맞춰 주요 지속가능...,single_hop_specific_query_synthesizer,삼성전자의 지속가능경영 전략과 주요 이슈 관리는 다음과 같이 이루어지고 있습니다.\...,[삼성전자 지속가능경영보고서 2024\n04\nOur Company Appendix...
2,What Samsung Global Code of Conduct mean for c...,[삼성전자 지속가능경영보고서 2024\n05\nOur Company Appendix...,Samsung Electronics established five core valu...,single_hop_specific_query_synthesizer,The Samsung Global Code of Conduct reflects th...,[Our Company\n04 CEO 메시지\n05 회사 소개 \n06 ...
3,What was the global operational footprint of S...,[Our Company AppendixMateriality Assessment Fa...,"By the end of 2023년, Samsung Electronics had a...",single_hop_specific_query_synthesizer,"By the end of 2023, Samsung Electronics had a ...",[삼성전자 지속가능경영보고서 2024\n05\nOur Company Appendix...
4,What are the 2023 financial results for the ma...,[주요 사업부\n Device eXperienceDX\nDS Device Solut...,The sales and operating profit figures for the...,single_hop_specific_query_synthesizer,The 2023 financial results for the main busine...,[DS Device Solutions\n메모리\n※ 상기 매출과 영업이익은 2023...


In [ ]:
# df.to_csv("report_2024_test.csv", index=False, encoding="utf-8-sig")

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_input          99 non-null     object
 1   reference_contexts  99 non-null     object
 2   reference           99 non-null     object
 3   synthesizer_name    99 non-null     object
 4   response            99 non-null     object
 5   retrieved_contexts  99 non-null     object
dtypes: object(6)
memory usage: 4.8+ KB


In [ ]:
# 데이터타입을 리스트로 변경해야함 (지금은 리스트모양의 str)
import ast
df["reference_contexts"] = df["reference_contexts"].apply(lambda x : ast.literal_eval(x))

In [41]:
from ragas import EvaluationDataset, evaluate
from ragas.metrics import Faithfulness, LLMContextRecall, FactualCorrectness
from ragas.llms import LangchainLLMWrapper

# ragas 평가
eval_llm = LangchainLLMWrapper(model)
dataset = EvaluationDataset.from_pandas(df)
dataset

C:\Users\user\AppData\Local\Temp\ipykernel_41104\3001053187.py:6: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  eval_llm = LangchainLLMWrapper(model)


EvaluationDataset(features=['user_input', 'retrieved_contexts', 'reference_contexts', 'response', 'reference'], len=99)

In [43]:
scores = evaluate(
    dataset,
    metrics = [Faithfulness(), LLMContextRecall(), FactualCorrectness()],
    llm = eval_llm
)

# Faithfulness() 높으면 할루시네이션 없다
# LLMContextRecall() : Retriever 수정
# FactualCorrectness() : Prompt 수정

Evaluating:   0%|          | 0/297 [00:00<?, ?it/s]

Exception raised in Job[132]: TimeoutError()


In [44]:
scores

{'faithfulness': 0.9187, 'context_recall': 0.8883, 'factual_correctness(mode=f1)': 0.5448}